<a href="https://colab.research.google.com/github/Goro-python/Autoencoder-mnist/blob/master/Sentiment_analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# NLP practice 

In [2]:
# sentiemnt classification task/

In [3]:
import tensorflow as tf
import numpy as np

In [4]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  9358k      0  0:00:08  0:00:08 --:--:-- 15.8M


In [8]:
!ls aclImdb/train/pos

0_9.txt       11876_10.txt  2501_8.txt	 4377_10.txt  625_10.txt   8127_8.txt
10000_8.txt   11877_10.txt  2502_8.txt	 4378_9.txt   6251_7.txt   8128_10.txt
10001_10.txt  11878_10.txt  2503_10.txt  4379_8.txt   6252_10.txt  8129_9.txt
10002_7.txt   11879_10.txt  2504_10.txt  437_9.txt    6253_8.txt   8130_10.txt
10003_8.txt   11880_10.txt  2505_9.txt	 4380_8.txt   6254_10.txt  813_10.txt
10004_8.txt   11881_9.txt   2506_9.txt	 4381_10.txt  6255_8.txt   8131_8.txt
10005_7.txt   11882_8.txt   2507_7.txt	 4382_8.txt   6256_8.txt   8132_10.txt
10006_7.txt   11883_8.txt   250_7.txt	 4383_9.txt   6257_10.txt  8133_7.txt
10007_7.txt   11884_8.txt   2508_10.txt  4384_10.txt  6258_8.txt   8134_10.txt
10008_7.txt   11885_7.txt   2509_9.txt	 4385_9.txt   6259_8.txt   8135_10.txt
1000_8.txt    11886_10.txt  2510_10.txt  4386_9.txt   6260_10.txt  8136_9.txt
10009_9.txt   11887_8.txt   251_10.txt	 4387_9.txt   6261_8.txt   8137_10.txt
10010_7.txt   11888_9.txt   2511_10.txt  4388_9.txt   6262_9.txt   

In [11]:
!cat aclImdb/train/pos/2500_9.txt

...would probably be the best word to describe this film (in my opinion). besides one great heck of a fan service for fan girls (well, that was redundant~), it was the story that blew me away. hurray for Takahisa Zeze and Gackt! and i know some people will disagree with me on this one, but it wasn't any of the big three actors (the guys that played Sho, son, and Kei) that gave the best performance (for me, anyway). it was Taro Yamamoto, boy #5 from battle Royale! <br /><br />don't get me wrong, i like Hyde, i worship his voice, but the problem was that some of his scenes came a little bit 'off', but i loved that scene where he danced with the dead guy's body killing the other guys. and Gackt wasn't at all that bad too, i preferred how his character was kind of aloof from the start. nothing much i can say about my background on lee-Hom Wang, i won't pretend to know him, but he carried his own weight with the star-studded cast. <br /><br />this movie paved the way for one of the best col

In [12]:
!rm -r aclImdb/train/unsup

In [17]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


In [20]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(5):
      print(text_batch.numpy()[i])
      print(label_batch.numpy()[i])

b'I am very disappointed with "K-911." The original "good" quality of "K-9" doesn\'t exist any more. This is more like a sitcom! Some of casts from original movie returned and got some of my memory back. The captain of Dooley now loves to hit him like a scene from old comedy show. That was crazy. What\'s the deal with the change of Police? It seems like they are now LAPD! Not San Diego PD. It is a completely different movie from "'
0
b"Giallo fans, seek out this rare film. It is well written, and full of all sorts of the usual low lifes that populate these films. I don't want to give anything away, so I wont even say anything about the plot. The whole movie creates a very bizarre atmosphere, and you don't know what to expect or who to suspect. Recommended! The only place I've seen to get this film in english is from European Trash Cinema, for $15."
1
b"Terry Gilliam's and David Peoples' teamed up to create one of the most intelligent and creative science fiction movies of the '90's. Pe

In [22]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

In [24]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

In [25]:
text_ds = raw_train_ds.map(lambda x, y: x)

In [26]:
vectorize_layer.adapt(text_ds)

In [29]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [30]:
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 125s 201ms/step - loss: 0.5080 - accuracy: 0.7103 - val_loss: 0.3057 - val_accuracy: 0.8776
Epoch 2/3
625/625 [==============================] - 122s 195ms/step - loss: 0.2281 - accuracy: 0.9101 - val_loss: 0.3232 - val_accuracy: 0.8774
Epoch 3/3
625/625 [==============================] - 121s 194ms/step - loss: 0.1136 - accuracy: 0.9595 - val_loss: 0.4341 - val_accuracy: 0.8702


In [32]:
model.evaluate(test_ds)

782/782 [==============================] - 34s 44ms/step - loss: 0.4509 - accuracy: 0.8625


[0.45092543959617615, 0.8625199794769287]

In [33]:
check_ds = "this is a great film"

In [36]:
# raw text model

In [39]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)





782/782 [==============================] - 34s 43ms/step - loss: 0.4509 - accuracy: 0.8625


<tf.Tensor 'text_vectorization_1/cond_2/Identity:0' shape=(None, 500) dtype=int64>

In [73]:
x = ["what a movie!","The junk ","awesome movie"]
myTensor = tf.convert_to_tensor(x, dtype=tf.string) 

print(myTensor)

end_to_end_model.predict(myTensor)


tf.Tensor([b'what a movie!' b'The junk ' b'awesome movie'], shape=(3,), dtype=string)


array([[0.50896865],
       [0.4643198 ],
       [0.6069356 ]], dtype=float32)